In [24]:
import json
import os
from datetime import datetime, timedelta

from crewai import Agent, Task, Crew, Process

from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import CSVSearchTool

import yfinance as yf

from IPython.display import Markdown

In [ ]:
os.environ['OPENAI_API_KEY'] = "MY_APY_KEY"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
csvWalletTool = CSVSearchTool(csv="./wallet.csv")

In [ ]:
customerManager = Agent(
    role="Customer Stocks Manager",
    goal="Get the customer question about the stock {ticket} and search the customer wallet CSV file for the stocks",
    backstory="""You're the manager of the customer investiments wallet.
    You are the client first contact and you provide the other analystis with 
    the necessary stock ticked and wallet information.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    tools=[csvWalletTool],
    allow_delegation=False,
    memory=True
)

In [ ]:
getCustomerWallet = Task(
    description="""Use the customer question and find the {ticket} in the CSV File.
    Provide if the stock is in the customer wallet and if it is, provide with the mean price he paid
    and the total numbers of stocks owned.
    """,
    expected_output="If the customer owns the stocks, provide the mean price paid and the total stock numbers.",
    agent=customerManager
)

In [ ]:
stockPriceAnalyst = Agent(
    role="Senior Stock Price Analyst",
    goal="Find the {ticket} stock price and analyses its trends. Compare with the price that the customer paid",
    backstory="""You're a highly experience in analyzing the price of specific stocks and
    make predictions about its future price
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [ ]:
# Criando yahoo finance tool
def fetch_stock_price(ticket):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)
    stock = yf.download(ticket, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return stock

yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks price for {ticket} from the last year about a specific company from the Yahoo Finance API.",
    func= lambda ticket: fetch_stock_price(ticket)
)

In [ ]:
fetch_stock_price("AAPL")

In [ ]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output="""Specify the current trend stocks price - Up, down or sideway.
    ex. stock='AAPL, price UP'.
    """,
    tools=[yahoo_finance_tool],
    agent=stockPriceAnalyst
)

In [ ]:
newsAnalyst = Agent(
    role="News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company.
    Provide a market Fear & Greed Index score about the company.
    For each requested stock asset, specify a number between 0 and 100, where 0 is extreme fear and
    100 is extreme greed.    
    """,
    backstory="""You're highly experience in analyzing market trends and news for more then 10 years.
    You're also a master level analyst in the human psychology.

    You understand the news, their title and information, but you look at those with a health dose of skeptcism.
    You consider the source of the news articles.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [ ]:
searchTool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [ ]:
getNews = Task(
    description=f"""Use the search tool to search news about the stock ticket.
    The current date is {datetime.now()}
    Compose the result into a helpfull report.
    """,
    expected_output="""A summary of the overall market and one sentence summary for the requested asset.
    Include the fear/greed score based on the news. Use format:
    <STOCK TICKET>
    <SUMMARY BASED ON NEWS>
    <FEAR/GREED SOCRE>
    """,
    agent=newsAnalyst,
    tools=[searchTool]
)

In [ ]:
stockRecommender = Agent(
    role="Chief Stock Analyst",
    goal="""Get the data from the customer currently stocks and provided input of stock price trends and
    the stock news to provide a recommendation: Buy, Sell or Hold the stock.
    """,
    backstory="""You're the leader of the stock analyst team. You have a great performance in the past 20 years in stock recommendation.
    With all your team information, you are able to provide the best recommendation for the customer to achieve
    the maximum value creation.
    """,
    verbose=True,
    llm=llm,
    allow_delegation=True,
    memory=True
)

In [ ]:
recommendStock = Task(
    description="""Use the stock price trend, the stock news report and the customer stock mean price of the {ticket} 
    to provide a recommendation: Buy, Sell or Hold. If the previous report are not well provided you can delegate back
    to specific analyst to work again in the their task.
    """,
    expected_output="""A brief paragraph with the summary of the reasons for recommendation and the 
    recommendation  it self in one of the three possible outputs: Buy, Sell or Hold. Use the format:
    <SUMMARY OF REASONS>
    <RECOMMENDATION>
    """,
    agent=stockRecommender,
    context=[getCustomerWallet, getStockPrice, getNews]
)

In [ ]:
copyWriter = Agent(
    role="Stock Content Writer",
    goal="""Write an insightfull compelling and informative 6 paragraph long newsletter
    based on the stock price report, the news report and the recommendation report.
    """,
    backstory="""You're a unbeliveble copy writer that understand complex financel concepts
    and explain for a dummie audience.
    You create compelling stories and narratives that resonate with the audience.
    """,
    verbose=False,
    llm=llm,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [ ]:
writeNewsletter = Task(
    description="""Use the stock price trend, the stock news report and the stock recommendation to write 
    an insightfull compelling and informative 6 paragraph long newsletter.
    Focus on the stock price trend, news, the fear/greed score and the summary reason for the recommendation.
    Include the recommendation in the newsletter.
    """,
    expected_output="""An eloquent 6 paragraph newsletter formated as Markdown in an easy readable manner.
    It should contain:
    - Introduction - set the overal picture.
    - Main part - provides the meat of the analysis including stock price trend, the news, the fear/greed score
    and the summary reason for the recommendation.
    - 3 bullets of the main summary reason of the recommendation.
    - Recommendation it self
    """,
    agent=copyWriter,
    context=[getStockPrice, getNews, recommendStock]
)

In [ ]:
crew = Crew(
    agents=[customerManager, stockPriceAnalyst, newsAnalyst, stockRecommender, copyWriter],
    tasks=[getCustomerWallet, getStockPrice, getNews, recommendStock, writeNewsletter],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15
)

In [ ]:
result = crew.kickoff(inputs={"ticket": "Give your thoughts about Amazon Stocks"})

In [ ]:
Markdown(result.raw)